In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL, StableDiffusionXLImg2ImgPipeline

from dreambooth.src import utils

In [ ]:
device = "cuda"

# fp16 cannot be run on CPU
dtype = torch.float16 if device == "cuda" else torch.float32

## Generate images with the stable diffusion xl base model and the LoRA weights

In [ ]:
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", 
    torch_dtype=dtype
)

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=dtype,
    variant="fp16",
    use_safetensors=True
)
pipe.to(device)
pipe.load_lora_weights(REPO_ID, weight_name="vuk_lora_weights.safetensors")

In [ ]:
prompt = "A photo of Vuk participating in a marathon."

sd_output = pipe(prompt=prompt, num_inference_steps=25, num_images_per_prompt=2)

In [ ]:
utils.show_images(sd_output.images)

## Add a refiner to improve image generation quality

In [ ]:
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    vae=vae,
    torch_dtype=dtype,
    variant="fp16",
    use_safetensors=True
)
refiner.to(device)

In [ ]:
refiner_output = refiner(
    prompt=prompt, 
    num_inference_steps=25, 
    generator=generator, 
    image=image
)

In [ ]:
refiner_output